In [21]:
from kaggle_environments import make, evaluate
import numpy as np
import random
import collections
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Создаем окружение для игры с заданным количеством ходов
env = make("rps", configuration={"episodeSteps": 100})

In [22]:
%%writefile rock_agent.py
# Пишем агентов:

# 1. Агент, который всегда выбирает камень
def rock_agent(observation, configuration):
    """
    Агент, который всегда выбирает камень (0).
    """
    return 0

Overwriting rock_agent.py


In [23]:
%%writefile paper_agent.py
# 2. Агент, который всегда выбирает бумагу
def paper_agent(observation, configuration):
    """
    Агент, который всегда выбирает бумагу (1).
    """
    return 1

Overwriting paper_agent.py


In [24]:
%%writefile scissors_agent.py
# 3. Агент, который всегда выбирает ножницы
def scissors_agent(observation, configuration):
    """
    Агент, который всегда выбирает ножницы (2).
    """
    return 2

Overwriting scissors_agent.py


In [25]:
%%writefile rock_paper_agent.py
# 4. Агент, который случайно выбирает между камнем и бумагой
import random

def rock_paper_agent(observation, configuration):
    """
    Случайно выбирает между камнем (0) и бумагой (1).
    """
    return random.randint(0, 1)

Overwriting rock_paper_agent.py


In [26]:
%%writefile rock_paper_scissors.py
# 5. Агент, который случайно выбирает между камнем и ножницами
import random

def rock_scissors_agent(observation, configuration):
    """
    Случайно выбирает между камнем (0) и ножницами (2).
    """
    return random.choice([0, 2])  # Выбор только между 0 и 2

Overwriting rock_paper_scissors.py


In [27]:
%%writefile paper_scissors_agent.py
# 6. Агент, который случайно выбирает между бумагой и ножницами
import random

def paper_scissors_agent(observation, configuration):
    """
    Случайно выбирает между бумагой (1) и ножницами (2).
    """
    return random.randint(1, 2)

Overwriting paper_scissors_agent.py


In [28]:
%%writefile rock_scissors_agent.py
# 7. Агент, который случайно выбирает между "камнем", "ножницами" и "бумагой"
import random

def rock_scissors_agent(observation, configuration):
    """
    Случайно выбирает 'камень' (0), 'ножницы' (2) или 'бумагу' (1).
    """
    return random.randint(0, 2)

Overwriting rock_scissors_agent.py


In [29]:
%%writefile copy_opponent_agent.py

# 8. Агент, который копирует последний ход оппонента
import random
def copy_opponent_agent(observation, configuration):
    """
    Копирует последний ход оппонента. Если ход первый, то выбирает случайно.
    """
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent_agent.py


In [30]:
%%writefile counter_reactionary_agent.py
# 9. Агент, который выбирает действие в зависимости от результатов последних ходов
import random
import math

last_counter_action = None
def get_score(left_move, right_move):
    """
    Возвращает счет на основе разницы между ходами.
    """
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

def counter_reactionary_agent(observation, configuration):
    """
    Если агент выиграл, выбирает действие, чтобы противодействовать.
    Иначе адаптируется к ходу противника.
    """
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)  # случайный первый ход
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs  # если выиграл, выбирает контр-ход
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs  # выбирает ход противника + 1

    return last_counter_action

Overwriting counter_reactionary_agent.py


In [31]:
%%writefile statistical_agent.py
# 10. Агент, который анализирует статистику ходов противника
import random
action_histogram = {}
def statistical_agent(observation, configuration):
    """
    Запоминает ходы противника и выбирает действие, чтобы противодействовать
    наиболее частому ходу противника.
    """
    global action_histogram
    if observation.step == 0:
        action_histogram = {}  # очищает историю в начале
        return random.randint(0, 2)  # случайный ход в начале
    
    action = observation.lastOpponentAction
    action_histogram[action] = action_histogram.get(action, 0) + 1  # считает ходы противника

    mode_action = max(action_histogram, key=action_histogram.get)  # находит наиболее частый ход
    return (mode_action + 1) % configuration.signs  # возвращает контр-ход

Overwriting statistical_agent.py


In [32]:
%%writefile Jonson_agent.py
#11. Агент Джонсона с анализом паттернов противника
import numpy as np
import collections
def Jonson_agent(observation, configuration):
    """
    Агент с анализом паттернов противника. Использует таблицу частот для предсказания.
    """
    k = 2
    global table, action_seq
    if observation.step % 25 == 0:
        action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])  # обновляет таблицу каждые 25 шагов
    if len(action_seq) <= 2 * k + 1:
        action = int(np.random.randint(3))
        if observation.step > 0:
            action_seq.extend([observation.lastOpponentAction, action])  # добавляет последний ход противника
        else:
            action_seq.append(action)
        return action

    key = ''.join([str(a) for a in action_seq[:-1]])  # ключ на основе последовательности ходов
    table[key][observation.lastOpponentAction] += 1  # обновляет таблицу по ключу

    action_seq[:-2] = action_seq[2:]
    action_seq[-2] = observation.lastOpponentAction
    key = ''.join([str(a) for a in action_seq[:-1]])
    
    if observation.step < 50:
        next_opponent_action_pred = np.argmax(table[key])  # предсказание по таблице
    else:
        scores = np.array(table[key])
        next_opponent_action_pred = np.random.choice(3, p=scores/scores.sum())
        
    action = (next_opponent_action_pred + 1) % 3
    if observation.step > 90:
        action = next_opponent_action_pred
    action_seq[-1] = action
    return int(action)

Overwriting Jonson_agent.py


In [33]:
%%writefile cycle_agent.py
import random

# Агент, который выбирает действия по кругу
def cycle_agent(observation, configuration):
    # Действия: 0 = "rock", 1 = "paper", 2 = "scissors"
    last_action = observation['lastOpponentAction'] if observation['step'] > 0 else random.randint(0, 2)
    return (last_action + 1) % 3

Overwriting cycle_agent.py


In [37]:
%%writefile aggressive_agent.py
import random
#13. Выбирает победное действие от последнего хода противника
def aggressive_agent(observation, configuration):
    # Агент выбирает действие, которое победит последнее действие противника
    last_opponent_action = observation['lastOpponentAction'] if observation['step'] > 0 else random.choice(["rock", "paper", "scissors"])
    if last_opponent_action == 0:  
        return 0
    elif last_opponent_action == 1:  
        return 1
    else:  
        return 2

Overwriting aggressive_agent.py


In [38]:
import numpy as np
from kaggle_environments import evaluate
import os
agents = ['rock_agent', 'paper_agent', 'scissors_agent',
          'copy_opponent_agent',  'cycle_agent',
          'counter_reactionary_agent', 'statistical_agent',
          'Jonson_agent', 'rock_paper_scissors',
          'rock_paper_agent', 'paper_scissors_agent',
          'rock_scissors_agent', 'aggressive_agent']

# Добавляем .py к каждому агенту
agents_py = [i + ".py" for i in agents]
n_agents = len(agents)

# Проверяем существование файлов
for agent_file in agents_py:
    if not os.path.exists(agent_file):
        raise FileNotFoundError(f"Файл агента '{agent_file}' не найден.")

# Инициализируем матрицу результатов
scores = np.zeros((n_agents, n_agents))

# Играем каждый агент против каждого
for i in range(n_agents):
    for j in range(i + 1, n_agents):
        result = evaluate("rps", [agents_py[i], agents_py[j]], configuration={"episodeSteps": 100}, num_episodes=1)
        scores[i][j] = result[0][0]  # сохраняем результат для агента i
        scores[j][i] = result[0][1]  # сохраняем результат для агента j

# Определяем победителя
vic = []
for i in range(n_agents):
    cnt_vic = sum(1 for j in range(n_agents) if scores[i][j] > 0)  # считаем победы агента
    vic.append(cnt_vic)

# Определяем индексы победителей
i_victors = [i for i, v in enumerate(vic) if v == max(vic)]
if len(i_victors) > 1:
    print("The winners are", '\n', *[agents_py[i] for i in i_victors], "!")
else:
    print("The winner is", agents_py[i_victors[0]], "!")

The winner is statistical_agent.py !


In [39]:
# Построение матрицы корреляций
df_scores = pd.DataFrame(scores, index=agents, columns=agents)
df_scores

,rock_agent,paper_agent,scissors_agent,copy_opponent_agent,cycle_agent,counter_reactionary_agent,statistical_agent,Jonson_agent,rock_paper_scissors,rock_paper_agent,paper_scissors_agent,rock_scissors_agent,aggressive_agent
rock_agent,0.0,-99.0,99.0,0.0,-97.0,-50.0,-99.0,-52.0,43.0,-47.0,0.0,0.0,0.0
paper_agent,99.0,0.0,-99.0,0.0,-99.0,-49.0,-97.0,-26.0,0.0,41.0,-52.0,0.0,0.0
scissors_agent,-99.0,99.0,0.0,0.0,-97.0,-49.0,-97.0,-31.0,-56.0,0.0,48.0,0.0,0.0
copy_opponent_agent,0.0,0.0,0.0,0.0,-49.0,99.0,-21.0,0.0,0.0,0.0,0.0,0.0,0.0
cycle_agent,97.0,99.0,97.0,49.0,0.0,-99.0,59.0,0.0,0.0,0.0,0.0,0.0,49.0
counter_reactionary_agent,50.0,49.0,49.0,-99.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0
statistical_agent,99.0,97.0,97.0,21.0,-59.0,0.0,0.0,0.0,28.0,35.0,46.0,0.0,21.0
Jonson_agent,52.0,26.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0
rock_paper_scissors,-43.0,0.0,56.0,0.0,0.0,0.0,-28.0,0.0,0.0,0.0,0.0,0.0,0.0
rock_paper_agent,47.0,-41.0,0.0,0.0,0.0,0.0,-35.0,-29.0,0.0,0.0,0.0,0.0,0.0
